In [1]:
import pandas as pd 
import numpy as np

In [2]:
talents=pd.read_csv('C:/Users/M-ODE/Desktop/Apziva/projects/3rd Project/potential_talents/data/potential_talents_CSV.csv')

In [3]:
talents_df=talents.copy()

In [4]:
talents_df

,id,job_title,location,connection,fit
0,1,2019 C.T. Bauer College of Business Graduate (...,"Houston, Texas",85,NaN
1,2,Native English Teacher at EPIK (English Progra...,Kanada,500+,NaN
2,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN
3,4,People Development Coordinator at Ryan,"Denton, Texas",500+,NaN
4,5,Advisory Board Member at Celal Bayar University,"İzmir, Türkiye",500+,NaN
...,...,...,...,...,...
99,100,Aspiring Human Resources Manager | Graduating ...,"Cape Girardeau, Missouri",103,NaN
100,101,Human Resources Generalist at Loparex,"Raleigh-Durham, North Carolina Area",500+,NaN
101,102,Business Intelligence and Analytics at Travelers,Greater New York City Area,49,NaN
102,103,Always set them up for Success,Greater Los Angeles Area,500+,NaN


In [5]:
talents_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 104 entries, 0 to 103
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   id          104 non-null    int64  
 1   job_title   104 non-null    object 
 2   location    104 non-null    object 
 3   connection  104 non-null    object 
 4   fit         0 non-null      float64
dtypes: float64(1), int64(1), object(3)
memory usage: 4.2+ KB


In [6]:
print('Total number of rows',len(talents_df))

Total number of rows 104


In [7]:
#check missing values
talents_df.isnull().sum()

id              0
job_title       0
location        0
connection      0
fit           104
dtype: int64

In [ ]:
# Preprocessing ; for old models/ Not advanced 
import re
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer

stop_words = set(stopwords.words("english"))
stemmer = PorterStemmer()
lemma = WordNetLemmatizer()

def preprocess_text(review):
    # Remove non-alphabet characters, convert to lowercase, and tokenize
    review = word_tokenize(re.sub('[^a-zA-Z]', ' ', review.lower()))
    # Remove stopwords
    review = [w for w in review if w not in stop_words]
    # Apply stemming 
    # review = [stemmer.stem(w) for w in review]
    # Apply lemmatization
    review = [lemma.lemmatize(w, pos='v') for w in review]
    # Join the processed words back into a string
    review = ' '.join(review)
    return review

talents_df['job_title_clean'] = talents_df['job_title'].apply(preprocess_text)
talents_df.head()


!pip install transformers 
!pip install torch 

## TF-IDF  

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

data = pd.read_csv('C:/Users/M-ODE/Desktop/Apziva/projects/3rd Project/potential_talents/data/potential_talents_CSV.csv')

# Keywords for ranking
keywords = ["Aspiring human resources", "seeking human resources"]

# Combine the job titles into a list for TF-IDF processing
job_titles = data['job_title'].tolist()

# Create a TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(job_titles)

# Calculate cosine similarities for each keyword
keyword_similarities = []
for keyword in keywords:
    keyword_tfidf = tfidf_vectorizer.transform([keyword])
    similarities = cosine_similarity(keyword_tfidf, tfidf_matrix)
    keyword_similarities.append(similarities[0])

# Calculate the average similarity for each job title based on all keywords
combined_scores = np.mean(keyword_similarities, axis=0)
data['fit'] = combined_scores
# Sort the DataFrame by similarity scores (higher scores indicate better matches)
ranked_data = data.sort_values(by='fit', ascending=False)

ranked_data.head()


,id,job_title,location,connection,fit
72,73,"Aspiring Human Resources Manager, seeking inte...","Houston, Texas Area",7,0.553403
45,46,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,0.539892
32,33,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,0.539892
16,17,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,0.539892
57,58,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,0.539892


## #Glovec

In [9]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\M-ODE\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [10]:
# Load DataFrame containing job titles
data = pd.read_csv('C:/Users/M-ODE/Desktop/Apziva/projects/3rd Project/potential_talents/data/potential_talents_CSV.csv')

# Tokenize the keywords
keywords = ["Aspiring human resources", "seeking human resources"]
tokenized_keywords = [word_tokenize(keyword.lower()) for keyword in keywords]


#Load Glove model
def load_glove_model(File):
    print("Loading Glove Model")
    glove_model = {}
    with open(File,'r', encoding='utf-8') as f:
        for line in f:
            split_line = line.split()
            word = split_line[0]
            embedding = np.array(split_line[1:], dtype=np.float64)
            glove_model[word] = embedding
    print(f"{len(glove_model)} words loaded!")
    return glove_model
embedding_model = load_glove_model('glove.6B.300d.txt')

# Process job_title column using NLTK 
tokenized_titles = [word_tokenize(title.lower()) for title in data['job_title']]

# Calculate the mean embedding for each keyword
keyword_embeddings = []
for keyword_tokens in tokenized_keywords:
    keyword_embedding = np.mean([embedding_model.get(word, np.zeros_like(embedding_model["a"])) for word in keyword_tokens], axis=0)
    keyword_embeddings.append(tuple(keyword_embedding))

Loading Glove Model
400001 words loaded!


In [11]:
# Calculate the embeddings for each candidate's job title
candidate_embeddings = []
for candidate_title in tokenized_titles:
    embedding_sum = np.zeros_like(embedding_model["a"])
    word_count = 0
    for word in candidate_title:
        if word in embedding_model:
            embedding_sum += embedding_model[word]
            word_count += 1
    if word_count > 0:
        candidate_embedding = embedding_sum / word_count
    else:
        candidate_embedding = np.zeros_like(embedding_model["a"])
    candidate_embeddings.append(candidate_embedding)
    
    
# Calculate similarity for each candidate
similarity_scores = []
for keyword_embedding in keyword_embeddings:
    candidate_similarities = []
    for candidate_embedding in candidate_embeddings:
        similarity = cosine_similarity([candidate_embedding], [keyword_embedding])[0, 0]
        candidate_similarities.append(similarity)
    similarity_scores.append(candidate_similarities)
    

# Combine the similarity scores for all keywords
combined_scores = np.mean(similarity_scores, axis=0)

# Check if the length of combined_scores matches the DataFrame length
if len(combined_scores) != len(talents_df):
    raise ValueError("Length of 'combined_scores' does not match the length of the DataFrame.")

# Update the 'fit' column in the DataFrame
data['fit'] = combined_scores


# Combine the similarity scores for all keywords
combined_scores = np.mean(similarity_scores, axis=0)

# Update the 'fit' column in the DataFrame
data['fit'] = combined_scores

# Sort the DataFrame based on the initial fit scores
data=data.sort_values(by='fit', ascending=False)

# View the resulting DataFrame
data.head()

,id,job_title,location,connection,fit
27,28,Seeking Human Resources Opportunities,"Chicago, Illinois",390,0.899883
29,30,Seeking Human Resources Opportunities,"Chicago, Illinois",390,0.899883
23,24,Aspiring Human Resources Specialist,Greater New York City Area,1,0.888238
5,6,Aspiring Human Resources Specialist,Greater New York City Area,1,0.888238
35,36,Aspiring Human Resources Specialist,Greater New York City Area,1,0.888238


## Bert

In [12]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from transformers import BertTokenizer, BertModel
import torch

# Load a pre-trained BERT model and tokenizer
model_name = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertModel.from_pretrained(model_name)

# Load DataFrame containing job titles
data = pd.read_csv('C:/Users/M-ODE/Desktop/Apziva/projects/3rd Project/potential_talents/data/potential_talents_CSV.csv')

# Keywords for ranking
keywords = ["Aspiring human resources", "seeking human resources"]

# Create an empty list to store the similarity scores for each keyword
keyword_similarities = []

# Tokenize job titles and obtain their BERT embeddings
job_title_tokens = [tokenizer(title, return_tensors="pt") for title in data['job_title']]
job_title_embeddings = []

for keyword in keywords:
    # Tokenize the current keyword and obtain its BERT embeddings
    keyword_tokens = tokenizer(keyword, return_tensors="pt")
    with torch.no_grad():
        keyword_embeddings = model(**keyword_tokens).last_hidden_state.mean(dim=1)
    
    # Calculate cosine similarities between the current keyword and job title embeddings
    similarities = []
    with torch.no_grad():
        for tokens in job_title_tokens:
            embeddings = model(**tokens).last_hidden_state.mean(dim=1)
            similarity = cosine_similarity(keyword_embeddings, embeddings).item()
            similarities.append(similarity)
    
    # Append the similarity scores for the current keyword to the list
    keyword_similarities.append(similarities)

# Combine or average the similarity scores based on all keywords for each job title
average_similarities = np.mean(keyword_similarities, axis=0)

# Add the average similarity scores to the DataFrame for ranking
data['fit'] = average_similarities

# Sort the DataFrame by similarity scores (higher scores indicate better matches)
ranked_data = data.sort_values(by='fit', ascending=False)

ranked_data.head()
# You can use ranked_data.tail() to view the bottom results as well


,id,job_title,location,connection,fit
29,30,Seeking Human Resources Opportunities,"Chicago, Illinois",390,0.879002
27,28,Seeking Human Resources Opportunities,"Chicago, Illinois",390,0.879002
98,99,Seeking Human Resources Position,"Las Vegas, Nevada Area",48,0.872550
2,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,0.848709
45,46,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,0.848709


In [ ]:
job_title_tokens 

## Sbert

In [13]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util

# Load a pre-trained SBERT model
model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# Load DataFrame containing job titles
data = pd.read_csv('C:/Users/M-ODE/Desktop/Apziva/projects/3rd Project/potential_talents/data/potential_talents_CSV.csv')

# Keywords for ranking
keywords = ["Aspiring human resources", "seeking human resources"]

# Create an empty list to store the similarity scores for each keyword
keyword_similarities = []

# Tokenize job titles and obtain their SBERT embeddings
job_title_embeddings = model.encode(data['job_title'].tolist(), convert_to_tensor=True)

for keyword in keywords:
    # Tokenize the current keyword and obtain its SBERT embedding
    keyword_embedding = model.encode(keyword, convert_to_tensor=True)
    
    # Calculate cosine similarities between the current keyword and job title embeddings
    similarities = util.pytorch_cos_sim(keyword_embedding, job_title_embeddings)
    
    # Convert the PyTorch tensor to a NumPy array before appending
    keyword_similarities.append(similarities.cpu().numpy())

# Combine or average the similarity scores based on all keywords for each job title
average_similarities = np.mean(keyword_similarities, axis=0)

# Reshape the average_similarities array to (104,)
average_similarities = average_similarities[0]

# Add the similarity scores to the DataFrame for ranking
data['fit'] = average_similarities.tolist()

# Sort the DataFrame by similarity scores (higher scores indicate better matches)
ranked_data = data.sort_values(by='fit', ascending=False)

ranked_data.head()


,id,job_title,location,connection,fit,similarity_score
29,30,Seeking Human Resources Opportunities,"Chicago, Illinois",390,NaN,0.901338
27,28,Seeking Human Resources Opportunities,"Chicago, Illinois",390,NaN,0.901338
98,99,Seeking Human Resources Position,"Las Vegas, Nevada Area",48,NaN,0.873473
32,33,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN,0.839355
2,3,Aspiring Human Resources Professional,"Raleigh-Durham, North Carolina Area",44,NaN,0.839355
